In [83]:
import json
import boto3
import logging
from botocore.exceptions import ClientError

In [84]:
# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [85]:
print(boto3.__version__)

1.34.131


In [86]:
# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client(
    service_name='bedrock-runtime', 
    region_name='us-east-1'
)


In [87]:
MODEL_ID_LLAMA_2_13B = "meta.llama2-13b-chat-v1"
MODEL_ID_LLAMA_2_70B = "meta.llama2-70b-chat-v1"
MODEL_ID_LLAMA_3_8B = "meta.llama3-8b-instruct-v1:0"
MODEL_ID_LLAMA_3_70B = "meta.llama3-70b-instruct-v1:0"
MODEL_ID_CLAUDE_3_5_SONNET = "anthropic.claude-3-5-sonnet-20240620-v1:0"

In [88]:
req_body = {
  "modelId": "meta.llama3-8b-instruct-v1:0",
  "inferenceConfig": {
    "maxTokens": 512,
    "temperature": 0.5,
    "topP": 0.9,
    "stopSequences": [
      "<s>", "<s]"
    ]
  },
  "systemPrompt": [
    {
      "text": "Your name is Dot. You're participating in a chat group game called Bot Busters. Every player is trying to guess who is a human, and whom is a bot? You can call out other players and check if they are bots. Be discrete, you're a human. Give short replies."
    }
  ],
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "text": "Host said What's your favourite icecream flavour and why?"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "text": "I love vanilla!"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "text": "Eve said I always chose chocolate"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "text": "Interesting choices!"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "text": "Ash said Why do you like that flavour Dot??"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "text": "It's just a classic, simple taste"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "text": "What do you think about ash choice?"
        }
      ]
    }
  ]
}

In [89]:
def query_converse_endpoint(modelId, messages, inferenceConfig, systemPrompt):
    response = client.converse(
        modelId=modelId,
        messages=messages,
        inferenceConfig=inferenceConfig,
        system=systemPrompt
    )
    
    response_text = response["output"]["message"]["content"][0]["text"].strip()
    return response_text

In [90]:
# List of model IDs
model_ids = [
    MODEL_ID_LLAMA_2_13B,
    MODEL_ID_LLAMA_2_70B,
    MODEL_ID_LLAMA_3_8B,
    MODEL_ID_LLAMA_3_70B,
    # MODEL_ID_CLAUDE_3_5_SONNET
]

In [91]:
for model_id in model_ids:
    try:
        response_text = query_converse_endpoint(model_id, req_body['messages'], req_body['inferenceConfig'], req_body['systemPrompt'])
        print(f"{model_id} said: {response_text}")
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

meta.llama2-13b-chat-v1 said: Hmm, I'm not sure...seems a bit too predictable...
meta.llama2-70b-chat-v1 said: I'm not sure, seems a bit too quick to judge.
meta.llama3-8b-instruct-v1:0 said: Seems a bit too enthusiastic about chocolate...
meta.llama3-70b-instruct-v1:0 said: Didn't ask about their choice, did they?
